# This is the scratch book for codes exploration

In [110]:
# from pathlib import Path
import sys
from pathlib import Path
import glob
import os
import shutil

# system PATH
code_path_str='/scratch/mr_proc'
sys.path.append(code_path_str)

# study PATH
code_dir = Path(code_path_str)
dataset_name = "PPMI"
dataset_path = Path('/scratch/0tmp/'+dataset_name)
dataset_out_path    = Path('/scratch/0tmp/'+dataset_name+'_SessionOrganized')
dataset_out_df_path = Path(code_dir / 'tab_data'  / 'PPMI_3T_sdMRI_3_07_2022_dcminfo.csv')

ppmi_img_dl_file  = code_dir / 'tab_data'  / 'PPMI_3T_sdMRI_3_07_2022.csv'  # Inormation from download database.
ppmi_img_dl_data  = pd.read_csv(ppmi_img_dl_file, sep=',')

In [115]:
def del_dir_safe(folder_):
    for filename in os.listdir(folder_):
        file_path = os.path.join(folder_, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
            
# Prepare dataset_dir
if not dataset_out_path.exists():
    os.makedirs(dataset_out_path)
else:  
    del_dir_safe(dataset_out_path)
    print(str(dataset_out_path)+' already exists, cleared!')
    
# Get subj list from raw dataset folder
sub_list = os.listdir(dataset_path)
dcm_df = pd.DataFrame(data={'Image Data ID':[], 'Visit':[], 'Subject':[], 'Modality':[], 'Image Date':[]})

for subj_ in sub_list:
    image_tab_df = ppmi_img_dl_data[ppmi_img_dl_data['Subject']==subj_].loc[:,['Image Data ID', 'Visit']].copy()
    # make new subject and session dir 
    sub_dir = dataset_out_path / subj_
    if not (sub_dir).exists():
        os.makedirs(sub_dir)
    else:
        print(subj_+' subject dir already exists!')
    
    # make new session dir
    for ses_str in image_tab_df.Visit.unique():
        ses_dir = dataset_out_path / subj_ / str(ses_str)
        if not ses_dir.exists():
            os.makedirs(ses_dir)
        else:
            print(subj_+' '+ 'session dir '+str(ses_str) + ' dir already exists!')
            
    # copy files from raw dataset folder to the new folder
    for modality_str_ in os.listdir(dataset_path / subj_):
        for date_str_ in os.listdir(dataset_path / subj_ / modality_str_):
            for img_str_ in os.listdir(dataset_path / subj_ / modality_str_ / date_str_):
                curr_ses=str(image_tab_df[image_tab_df['Image Data ID']==img_str_].iloc[0,1])
                alldcm = glob.glob(str(dataset_path / subj_ / modality_str_ / date_str_ / img_str_)+'/*')
                source_dir = str(dataset_path / subj_ / modality_str_ / date_str_ / img_str_)
                target_dir = str(sub_dir / curr_ses / img_str_)
                subj_dcm_dict = {'Image Data ID':[img_str_], 'Visit':[curr_ses], 'Subject':[subj_], 'Modality':[modality_str_], 'Image Date':[date_str_]}
                dcm_df=pd.concat([dcm_df, pd.DataFrame(data=subj_dcm_dict)])
                os.makedirs(target_dir)
                [shutil.copy2(dcm_file_, target_dir) for dcm_file_ in alldcm]
                #print(subj_+' '+modality_str_+' '+date_str_+' '+img_str_+' copied!')

/scratch/0tmp/PPMI_SessionOrganized already exists, cleared!


In [ ]:
if not dataset_out_df_path.exists():
    dcm_df.to_csv(dataset_out_df_path)
else:
    os.unlink(dataset_out_df_path)
    dcm_df.to_csv(dataset_out_df_path)
dcm_df

,Image Data ID,Visit,Subject,Modality,Image Date
0,I787960,0,01218AUG16,DTI,2016-08-18_01_04_43.0
0,I787959,0,01218AUG16,MPRAGE_Phantom_GRAPPA2,2016-08-18_00_59_00.0
0,I353355,1,3101,Axial_PD-T2_TSE,2012-08-17_11_23_13.0
0,I449541,7,3101,Axial_PD-T2_TSE_FS,2014-08-20_10_57_57.0
0,I353354,1,3101,DTI_gated,2012-08-17_11_34_08.0
...,...,...,...,...,...
0,I253992,1,3115,DTI_gated,2011-08-01_14_02_21.0
0,I337531,5,3115,DTI_gated,2012-07-18_14_35_16.0
0,I337533,5,3115,DTI_gated,2012-07-18_15_01_19.0
0,I253995,1,3115,MPRAGE_GRAPPA,2011-08-01_13_30_13.0
